# Imports 

In [63]:
from glob import glob
import PIL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, f1_score, log_loss

import torch
from torchvision import transforms as T
from transformers import AutoModelForImageClassification, AutoImageProcessor

# Hugging Face Hub

In [82]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /mnt/c/Users/Mustapha/.cache/huggingface/token
Login successful


# Data

In [3]:
train_df = pd.read_csv('../data/train.csv')

In [4]:
# glob, test folder
test_paths = [pth.split("/")[-1] for pth in glob('../data/test/*')]
test_df = pd.DataFrame(sorted(test_paths, key=lambda x: int(x.split(".")[0])))

### Splits

In [71]:
seed= 42
train_df, validation_df = train_test_split(
    train_df, test_size=0.1, stratify=train_df["label"].values, random_state=seed
)
validation_df, holdout_df = train_test_split(
    validation_df, test_size=0.5, stratify=validation_df["label"].values, random_state=seed
)

In [72]:
train_df.reset_index(drop=True, inplace=True)
validation_df.reset_index(drop=True, inplace=True)
holdout_df.reset_index(drop=True, inplace=True)

### Data loaders

In [73]:
label2id = {"NO_AI": 0, "AI": 1}
id2label = {0: "NO_AI", 1: "AI"}

class data(torch.utils.data.Dataset):
    def __init__(self, train_labels: pd.DataFrame = train_df, split_name:str = 'train', aug_transforms=None):
        self.train_labels = train_labels
        self.index = train_labels.index
        self.split_name = split_name
        self.aug_transforms = aug_transforms
        
    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
            
        try :
            name = self.train_labels.loc[index, "id"]
            if self.split_name == "train":
                label = self.train_labels.loc[index, "label"]    
        except IndexError:
            raise IndexError('Index out of range')
        
        path = f'../data/{self.split_name}/{name}'
        # image = plt.imread(path)
        image = PIL.Image.open(path)
        image = self.aug_transforms(image)
        
        if self.split_name == "train":
            return {"img_path": path, "image": image, "label": label}
        else:
            return {"img_path": path, "image": image}
    
    def info(self):
        print(f'Number of images: {len(self)}')
        print(f'Classes: {self.train_labels["label"].unique()}')
        print(f'Images shape : {self[0][0].shape}')
    

### model

In [74]:
model_ckpt = "microsoft/swin-tiny-patch4-window7-224"
model = AutoModelForImageClassification.from_pretrained(
    model_ckpt, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True,
)
image_processor = AutoImageProcessor.from_pretrained(model_ckpt)

loading configuration file config.json from cache at /mnt/c/Users/Mustapha/.cache/huggingface/hub/models--microsoft--swin-tiny-patch4-window7-224/snapshots/83d40fb5b9320b349382208d9e7fe998484e99df/config.json
Model config SwinConfig {
  "_name_or_path": "microsoft/swin-tiny-patch4-window7-224",
  "architectures": [
    "SwinForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    6,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 96,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "NO_AI",
    "1": "AI"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "AI": 1,
    "NO_AI": 0
  },
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swin",
  "num_channels": 3,
  "num_heads": [
    3,
    6,
    12,
    24
  ],
  "num_layers": 4,
  "out_features": null,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "stage_names": [

### transformations

In [75]:
mean, std = image_processor.image_mean, image_processor.image_std
size = image_processor.size["height"]


train_transforms = T.Compose([
    T.RandomResizedCrop(size),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])
test_transforms = T.Compose([
    # T.Resize(256),
    T.CenterCrop(size),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

In [76]:
train_dataset = data(train_df, split_name='train', aug_transforms=train_transforms)
validation_dataset = data(validation_df, split_name='train', aug_transforms=test_transforms)
holdout_dataset = data(holdout_df, split_name='train', aug_transforms=test_transforms)
test_dataset = data(test_df, split_name='test', aug_transforms=test_transforms)

# Training

In [77]:
from transformers import TrainingArguments


model_name = model_ckpt.split("/")[-1]
batch_size = 64

args = TrainingArguments(
    f"{model_name}-aiornot-simple",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=3,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [78]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    logloss = log_loss(labels, preds)
    return {"accuracy": acc, "f1": f1, "log_loss": logloss}

In [79]:
def collate_fn(examples):
    pixel_values = torch.stack([example["image"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [80]:
from transformers import Trainer 


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [81]:
train_results = trainer.train()

/home/mus5900/anaconda3/envs/nlp_2/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15080
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 4
  Total optimization steps = 590
  Number of trainable parameters = 27520892


RuntimeError: CUDA out of memory. Tried to allocate 294.00 MiB (GPU 0; 2.00 GiB total capacity; 1.41 GiB already allocated; 0 bytes free; 1.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Evaluation

In [ ]:
trainer.evaluate(holdout_dataset)